In [ ]:
import multiprocessing as mp
import pandas as pd
import numpy as np
from multiprocessing import  Pool

In [ ]:
df = pd.read_csv('common_passwords.csv')

In [ ]:
df.head()

,password,length,num_chars,num_digits,num_upper,num_lower,num_special,num_vowels,num_syllables
0,123456,6,0,6,0,0,0,0,1
1,password,8,8,0,0,8,0,2,2
2,12345678,8,0,8,0,0,0,0,1
3,qwerty,6,6,0,0,6,0,1,3
4,123456789,9,0,9,0,0,0,0,1


In [ ]:
df.isnull().any()

password         False
length           False
num_chars        False
num_digits       False
num_upper        False
num_lower        False
num_special      False
num_vowels       False
num_syllables    False
dtype: bool

In [ ]:
def imputasi(df_input):
  list_columns = df_input.columns
  class_column = list_columns[-9]
  for column in list_columns[:-9]:
    df_input[column] = df_input[column].fillna(df_input.groupby(class_column)[column].transform('mean'))
  return df_input

In [ ]:
from dataclasses import dataclass
def parallelize_dataframe(data, func, n_cores=4):
    df_split = np.array_split(data, n_cores)
    pool = Pool(n_cores)
    data = (pool.map(func, df_split))
    pool.close()
    pool.join()
    return data

In [ ]:
parallelize_dataframe(df,imputasi,n_cores=4)

[       password  length  num_chars  num_digits  num_upper  num_lower  \
 0        123456       6          0           6          0          0   
 1      password       8          8           0          0          8   
 2      12345678       8          0           8          0          0   
 3        qwerty       6          6           0          0          6   
 4     123456789       9          0           9          0          0   
 ...         ...     ...        ...         ...        ...        ...   
 2495     santos       6          6           0          0          6   
 2496     rrrrrr       6          6           0          0          6   
 2497      randy       5          5           0          0          5   
 2498    picture       7          7           0          0          7   
 2499     payton       6          6           0          0          6   
 
       num_special  num_vowels  num_syllables  
 0               0           0              1  
 1               0        

## **Modelling KNN**


## **Membagi** data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(df, test_size=0.2)

In [ ]:
print(data_latih.shape)
print(data_uji.shape)

(8000, 9)
(2000, 9)


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('password')

In [ ]:
label_uji = data_uji.pop('password')

## Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [ ]:
def jarakEu(data1, data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [ ]:
jarak = jarakEu(data_latih.iloc[0], data_uji.iloc[0])
jarak

10.63014581273465

## Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

# Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [ ]:
from collections import Counter

def knn(k, datalatih, labellatih, datauji):
  jarak = np.array([jarakEu(df.iloc[x], datauji) for x in range (datalatih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [ ]:
hasil = knn(3, data_latih, label_latih, data_uji.iloc[0])
hasil

'spot'

##  KNN pada Scikit-learn

Selain secara manual, Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 3)
model_fit = KNN.fit(data_latih, label_latih)
kelas = model_fit.predict([data_uji.iloc[0]])
print(kelas)

['01011975']


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
